In [1]:
import pandas as pd
import numpy as np
import os
import dask

In [2]:
s3_monitor_dir = '/Users/brianm/Dropbox/cc_AuroraInsight/data/s3/s3_monitor/'
s3_results_dir = '/Users/brianm/Dropbox/cc_AuroraInsight/data/s3'

s3_results_file = 's3_scan3.csv'
s3_results_file_path = os.path.join(s3_results_dir, s3_results_file)

In [3]:
file_list = os.listdir(s3_monitor_dir)
file_list_2 = os.listdir(os.path.join(s3_monitor_dir, 's3_monitor'))
file_list.remove('s3_monitor')

In [4]:
len(file_list)

412471

In [5]:
file_list = [name for name in file_list 
             if '.swp' not in name and 
             os.path.isfile(os.path.join(s3_monitor_dir, name))]


In [6]:
len(file_list)

412470

In [7]:
s3_monitor_lists = []




In [8]:
# 15 is GET size
discard_idx = [6, 13,14,16,17, 18]
def line_cleanup(line):
    '''
    Lines have a variable amount of data at line 19 depending on operation
    Collapse 19 until next '-' into one element
    '''
    if len(line) == 25:
        # Handle TRANSITION (25 elemements)
        #print(line)
        line.pop(6)
        new_line = line[:20]
        new_line[8] = 'TRANSITION'
        #print(new_line)
       
    else:    
        # Handle GET (32 elem), PUT (30 elem) 
        new_line = []
        at_19 = False
        line_19 = None
        for idx, elem in enumerate(line):
            if idx in discard_idx:
                continue
            if idx == 2:
                elem = elem.replace('[','')
            if idx == 3:
                elem = elem.replace(']','')
            #print(idx, elem)
            if idx == 19:
                at_19 = True
            if at_19 == True:
                if elem == '-':
                    #print('1')
                    #print(line_19)
                    new_line.append(line_19)
                    at_19 = False
                else:
                    #print('2')
                    #print(line_19, elem)
                    if line_19:
                        line_19 += '|' + elem
                    else:
                        line_19 = elem

                continue
            new_line.append(elem)
    return new_line
            
            

In [9]:
columns = ['id', 'bucket', 'time', 'time_ms?','accessw_ip', 
           'arn', 'full_action', 'key', 'basic_action', 'key_extended', 
           'http', 'response', 'size', 'software_method',
           'token', 'sig_version',  'encoding', 'header', 's3_location',
           'aws_version'
          ]
needed_columns =  ['bucket', 'time', 'accessw_ip', 
          'full_action', 'key', 'basic_action', 'size', 
            'software_method'
                  ]



In [10]:
def reformat_info(list_of_lines):
    '''
    There are three types 
    '''
    
    df_lines = pd.DataFrame(list_of_lines, columns=columns)
    df_results = df_lines[needed_columns].copy()
    df_results['file'] = df_results['key'].map(lambda x: os.path.split(x)[1])
    df_results['prefix'] = df_results['key'].map(lambda x: os.path.split(x)[0])
    return df_results

In [11]:

first_time = True
for file_name in file_list:
    with open(os.path.join(s3_monitor_dir, file_name)) as s3_monitor_file:
        content = s3_monitor_file.readlines()
    content_2 = [line.strip().split() for line in content] 
    content_3 = [line_cleanup(line) for line in content_2]
    #print(content)
    df_content = reformat_info(content_3)
    if '-' in df_content['size'].values:
        print(df_content.head())
        break
    #print(df_content.head())

    if first_time:
        first_time = False
        df_content.to_csv(s3_results_file_path, mode='w', header=True)
        
            
    df_content.to_csv(s3_results_file_path, mode='a', header=False)


  
    
    

     bucket                  time      accessw_ip      full_action key  \
0  aii-data  23/Sep/2020:22:43:18  34.204.179.167  REST.GET.BUCKET   -   

  basic_action size                                    software_method file  \
0         "GET    -  "Boto3/1.14.48|Python/3.6.12|Linux/4.14.177-10...    -   

  prefix  
0         


In [12]:
df_content['size']

0    -
Name: size, dtype: object

In [13]:
content

['cce8acfe91eb3a7432b8a2f8b610b830625a027c6c47dd3436ae147a71b71d16 aii-data [23/Sep/2020:22:43:18 +0000] 34.204.179.167 arn:aws:sts::874847117436:assumed-role/aii-lambda/compile-collection-catalog E095F2030E608DA6 REST.GET.BUCKET - "GET /?prefix=v1%2Fdev%2Fterrestrial%2FL0%2FOmaha%2F&delimiter=%2F&encoding-type=url HTTP/1.1" 200 - 492 - 37 36 "-" "Boto3/1.14.48 Python/3.6.12 Linux/4.14.177-104.253.amzn2.x86_64 exec-env/AWS_Lambda_python3.6 Botocore/1.17.48" - vp8LM1Hj1HNeh1QVk9AOvGm1ThcsBeH0JLYUkov/OFjJ0tj8j/63kHfwNEUUxI2h0FBuDs4gXHE= SigV4 ECDHE-RSA-AES128-GCM-SHA256 AuthHeader aii-data.s3.amazonaws.com TLSv1.2\n']

In [14]:
content_2

[['cce8acfe91eb3a7432b8a2f8b610b830625a027c6c47dd3436ae147a71b71d16',
  'aii-data',
  '[23/Sep/2020:22:43:18',
  '+0000]',
  '34.204.179.167',
  'arn:aws:sts::874847117436:assumed-role/aii-lambda/compile-collection-catalog',
  'E095F2030E608DA6',
  'REST.GET.BUCKET',
  '-',
  '"GET',
  '/?prefix=v1%2Fdev%2Fterrestrial%2FL0%2FOmaha%2F&delimiter=%2F&encoding-type=url',
  'HTTP/1.1"',
  '200',
  '-',
  '492',
  '-',
  '37',
  '36',
  '"-"',
  '"Boto3/1.14.48',
  'Python/3.6.12',
  'Linux/4.14.177-104.253.amzn2.x86_64',
  'exec-env/AWS_Lambda_python3.6',
  'Botocore/1.17.48"',
  '-',
  'vp8LM1Hj1HNeh1QVk9AOvGm1ThcsBeH0JLYUkov/OFjJ0tj8j/63kHfwNEUUxI2h0FBuDs4gXHE=',
  'SigV4',
  'ECDHE-RSA-AES128-GCM-SHA256',
  'AuthHeader',
  'aii-data.s3.amazonaws.com',
  'TLSv1.2']]

In [15]:
content_2[0]

['cce8acfe91eb3a7432b8a2f8b610b830625a027c6c47dd3436ae147a71b71d16',
 'aii-data',
 '[23/Sep/2020:22:43:18',
 '+0000]',
 '34.204.179.167',
 'arn:aws:sts::874847117436:assumed-role/aii-lambda/compile-collection-catalog',
 'E095F2030E608DA6',
 'REST.GET.BUCKET',
 '-',
 '"GET',
 '/?prefix=v1%2Fdev%2Fterrestrial%2FL0%2FOmaha%2F&delimiter=%2F&encoding-type=url',
 'HTTP/1.1"',
 '200',
 '-',
 '492',
 '-',
 '37',
 '36',
 '"-"',
 '"Boto3/1.14.48',
 'Python/3.6.12',
 'Linux/4.14.177-104.253.amzn2.x86_64',
 'exec-env/AWS_Lambda_python3.6',
 'Botocore/1.17.48"',
 '-',
 'vp8LM1Hj1HNeh1QVk9AOvGm1ThcsBeH0JLYUkov/OFjJ0tj8j/63kHfwNEUUxI2h0FBuDs4gXHE=',
 'SigV4',
 'ECDHE-RSA-AES128-GCM-SHA256',
 'AuthHeader',
 'aii-data.s3.amazonaws.com',
 'TLSv1.2']

In [16]:
len(content_2[0])

31

In [17]:


for file_name in file_list_2:
    with open(os.path.join(os.path.join(s3_monitor_dir, 's3_monitor', file_name))) as s3_monitor_file:
        content = s3_monitor_file.readlines()
    content_2 = [line.strip().split() for line in content] 
    content_3 = [line_cleanup(line) for line in content_2]
    #print(content)
    df_content = reformat_info(content_3)
    #print(df_content.head())   
    if '-' in df_content['size'].values:
        print(df_content.head())
        break
    df_content.to_csv(s3_results_file_path, mode='a', header=False)

     bucket                  time      accessw_ip      full_action  \
0  aii-data  02/Oct/2020:01:05:31  34.201.103.201  REST.GET.OBJECT   
1  aii-data  02/Oct/2020:01:05:32  34.201.103.201  REST.GET.OBJECT   
2  aii-data  02/Oct/2020:01:05:33  34.201.103.201  REST.GET.OBJECT   
3  aii-data  02/Oct/2020:01:05:33  34.201.103.201  REST.GET.OBJECT   
4  aii-data  02/Oct/2020:01:05:34  34.201.103.201  REST.GET.OBJECT   

                                                 key basic_action        size  \
0  v1/dev/terrestrial/L0/RuralMichigan/20200928/2...         "GET  3103784960   
1  v1/dev/terrestrial/L0/RuralMichigan/20200928/2...         "GET  3103784960   
2  v1/dev/terrestrial/L0/RuralMichigan/20200928/2...         "GET  3103784960   
3  v1/dev/terrestrial/L0/RuralMichigan/20200928/2...         "GET  3103784960   
4  v1/dev/terrestrial/L0/RuralMichigan/20200928/2...         "GET  3103784960   

                                     software_method  \
0  "Boto3/1.15.10|Python/3.7.9|Linux

In [18]:
test = [line_cleanup(line) for line in content_2]

In [19]:
test[0]

['cce8acfe91eb3a7432b8a2f8b610b830625a027c6c47dd3436ae147a71b71d16',
 'aii-data',
 '02/Oct/2020:01:05:31',
 '+0000',
 '34.201.103.201',
 'arn:aws:sts::874847117436:assumed-role/Michigan_ecs_instance_role/i-01024fd2a1e070393',
 'REST.GET.OBJECT',
 'v1/dev/terrestrial/L0/RuralMichigan/20200928/2020-09-28T08-12-27_31898A7_C2A47E1729_RuralMichigan/iq_out_RX2_31898A7_0_1601301618.sigmf-data',
 '"GET',
 '/v1/dev/terrestrial/L0/RuralMichigan/20200928/2020-09-28T08-12-27_31898A7_C2A47E1729_RuralMichigan/iq_out_RX2_31898A7_0_1601301618.sigmf-data',
 'HTTP/1.1"',
 '206',
 '3103784960',
 '"Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149.317.amzn2.x86_64|exec-env/AWS_ECS_EC2|Botocore/1.18.10|Resource"',
 'nM6h1oghJOL596w2dciNnFKK06UST1vrY/+lvsTmekOHHNNP4boZQbuISlq83+QdwH77x32dZZk=',
 'SigV4',
 'ECDHE-RSA-AES128-GCM-SHA256',
 'AuthHeader',
 'aii-data.s3.amazonaws.com',
 'TLSv1.2']

In [20]:
df_content = reformat_info(test)

In [21]:
df_content

,bucket,time,accessw_ip,full_action,key,basic_action,size,software_method,file,prefix
0,aii-data,02/Oct/2020:01:05:31,34.201.103.201,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601301618.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
1,aii-data,02/Oct/2020:01:05:32,34.201.103.201,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601301618.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
2,aii-data,02/Oct/2020:01:05:33,34.201.103.201,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601301618.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
3,aii-data,02/Oct/2020:01:05:33,34.201.103.201,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601301618.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
4,aii-data,02/Oct/2020:01:05:34,34.201.103.201,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601301618.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
...,...,...,...,...,...,...,...,...,...,...
69,aii-data,02/Oct/2020:01:57:42,100.24.121.22,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601405000.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
70,aii-data,02/Oct/2020:01:57:42,100.24.121.22,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601405000.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
71,aii-data,02/Oct/2020:01:57:43,100.24.121.22,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601405000.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
72,aii-data,02/Oct/2020:01:57:44,100.24.121.22,REST.GET.OBJECT,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...,"""GET",3103784960,"""Boto3/1.15.10|Python/3.7.9|Linux/4.14.193-149...",iq_out_RX2_31898A7_0_1601405000.sigmf-data,v1/dev/terrestrial/L0/RuralMichigan/20200928/2...
